In [43]:
from smodels.theory import branch,element,particle,model
from smodels.experiment.databaseObj import Database
from smodels.share.models import mssm
from smodels.experiment.finalStateParticles import finalStates
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.auxiliaryFunctions import index_bisect, simParticles
import numpy as np
import sys,time

In [3]:
slhafile = 'inputFiles/slha/lightEWinos.slha'
bsmModel = model.Model(inputFile=slhafile, BSMparticles=BSMList, SMparticles=SMList)
bsmModel.updateParticles()

BSMparticles = bsmModel.BSMparticles
SMparticles = bsmModel.SMparticles
fStates = finalStates.SMparticles

In [29]:
allParticles = []
for p in SMparticles+BSMparticles+fStates:
    if any(p is pB for pB in allParticles):
        continue
    allParticles.append(p)
allParticles = sorted(allParticles, key = lambda p: id(p))
idList = [id(p) for p in allParticles]

In [35]:
#Now create huge comparison matrix with all unique states, including inclusive ones:
compMatrix = np.zeros((len(allParticles),len(allParticles)))
for i,p1 in enumerate(allParticles):
    for j,p2 in enumerate(allParticles):
        if i <= j:
            continue
        compMatrix[i,j] = int(p1 == p2)
compMatrix = compMatrix + compMatrix.T + np.identity(len(allParticles))

0.14673185348510742


In [48]:
#Directly compare all particles with each other:
t0 = time.time()
for p1,p2 in zip(allParticles,allParticles):
    i = (p1 == p2)
print(time.time()-t0)

0.0013458728790283203


In [49]:
#Compare all particles with each other using compMatrix and idList:
t0 = time.time()
for p1,p2 in zip(allParticles,allParticles):
    i1 = index_bisect(idList, id(p1))
    i2 = index_bisect(idList, id(p2))
    i = compMatrix[i1,i2]
print(time.time()-t0)

0.0009138584136962891


In [45]:
#Compare two unordered sets of particles:
l1 = allParticles[20:][::4]
l2 = allParticles[:45][::2][:len(l1)]
print(l1)
print(l2)

[A0~, su_L~, sc_L~, st_1~, N2, sta_2, se_L, snta_L, sb_1, ss_L, sc_R, ta, nu, u, t+, W+, u, t-, ta+, g~, higgs~, l+, W, L, *, HSCP, RHadronQ]
[se_L~, se_R~, snmu_L~, sta_1~, sta_2~, N1~, N3~, C1-, H+, H0, A0~, C2+, su_L~, su_R~, sc_L~, sc_R~, st_1~, st_2~, N2, sta_1, sta_2, snmu_L, se_L]


In [46]:
t0 = time.time()
print(simParticles(l1,l2))
print(time.time()-t0)

False
0.0013554096221923828


In [54]:
t0 = time.time()
l1Index = [index_bisect(idList,id(p)) for p in l1]
l2Index = [index_bisect(idList,id(p)) for p in l2]
print(all(compMatrix[i1,i2] for i1 in l1Index for i2 in l2Index))
print(time.time()-t0)

False
0.0015981197357177734
